In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime 
from datetime import date
import math
import pandas_datareader as web
import yfinance as yf
import time  # Import the time module
import os  # Import the os module
from sklearn.model_selection import TimeSeriesSplit

In [31]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, GRU
from keras.layers import Dropout

In [4]:
# reshape 2D dataset (samples + features) en 3D (samples/time step + features + séquence) 
# en ajoutant la dimension fenêtre/séquence (longueur est window_size) N, pour prédire N+1. 
# y_pos est la position de colonnes du target. Normalament la prémière colonne
# dataset.shape[0] est le nombre des échantillons
def rnn_3D_data(dataset, window_size, y_pos=0): 
    X = []
    y = []
    for i in range(window_size, dataset.shape[0]):
        X.append(dataset[i-window_size: i])
        y.append(dataset[i, y_pos])
    X, y = np.array(X), np.array(y)
    return X, y

In [5]:
def data_retrieve(stock, start_date, end_date): 

# Define the list of tickers for the stocks you want to fetch data for
    tickers = [stock]

    # Define the list of stock indices and other features
    indices = ['^GSPC', '^DJI', '^IXIC', 'DX-Y.NYB', '^GDAXI', '^FTSE', '^N225', '^VIX']
    commodities = ['GC=F', 'BZ=F']
    forex = ['EURUSD=X', 'JPY=X']

    symbols = indices + commodities + forex
    # Fetch the historical data using yfinance for tickers (OHLC and Volume)
    ticker_data = yf.download(tickers, start=start_date, end=end_date)

    # Fetch the historical data using yfinance for indices and commodities (Close Price)
    close_data = yf.download(symbols, start=start_date, end=end_date)['Adj Close']

# Concatenate the close_data with the ticker_data to include Close Price for all symbols
    all_data = pd.concat([ticker_data, close_data], axis=1)

# Print the data
    return all_data

In [32]:
# Définir le modèle CNN peu profond e 6 couches, sur l'architecture VGG
def hyper_GRU_M0(data_training, data_testing, win_size, num_epoch, num_cell, batch_size, dropout):

    scalar = MinMaxScaler()

    data_training_scaled = scalar.fit_transform(data_training)

    X_train, y_train = rnn_3D_data(data_training_scaled, window_size=win_size, y_pos=4)
    # add the winodw before test data
    data_test = pd.concat([data_training.tail(win_size), data_testing], ignore_index=True)
    data_test_scaled = scalar.fit_transform(data_test)
    X_test, y_test = rnn_3D_data(data_test_scaled, window_size=win_size, y_pos=4)

    model = Sequential()

    model.add(GRU(units = num_cell, input_shape = (X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(dropout))
    model.add(Dense(units = 1))

    start_time = time.time()
    # Compiling the RNN
    model.compile(optimizer = 'adam', loss = 'mean_squared_error')
    lstm_training = model.fit(X_train, y_train, epochs=num_epoch, batch_size = batch_size,\
                        shuffle=False, validation_data=(X_test, y_test),verbose=1)     
    
    end_time = time.time()
    execution_time = end_time - start_time
    
    # evaluer le modèle
    # transforme MSE en RMSE
    loss_history = pd.DataFrame(lstm_training.history).pow(.5)
    loss_history.index += 1
#calcule la valeur minimale de la colonne val_loss dans le DataFrame loss_history, 
#ce qui correspond au meilleur RMSE de validation atteint par le modèle.
    best_rmse = loss_history.val_loss.min()
    
    resultat = ['GRU_M0_Sp',win_size,num_cell, num_epoch, batch_size, dropout, round(best_rmse, 4), round(execution_time, 0)]
    return resultat

# 1. Raw data collection

In [7]:
start_date = datetime.datetime(2010, 1, 2)
end_date = datetime.datetime.today()

In [8]:
raw_data = yf.download('^GSPC', start=start_date, end=end_date)
raw_data.to_parquet('sp500_data.parquet')

[*********************100%***********************]  1 of 1 completed


In [9]:
raw_data.to_parquet('sp500_data.parquet')

In [10]:
data = pd.read_parquet('sp500_data.parquet')

In [11]:
print('There are {} number of days in the dataset.'.format(data.shape[0]))

There are 3430 number of days in the dataset.


In [12]:
df = data.copy()
df = df.dropna()
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,1116.560059,1133.869995,1116.560059,1132.989990,1132.989990,3991400000
2010-01-05,1132.660034,1136.630005,1129.660034,1136.520020,1136.520020,2491020000
2010-01-06,1135.709961,1139.189941,1133.949951,1137.140015,1137.140015,4972660000
2010-01-07,1136.270020,1142.459961,1131.319946,1141.689941,1141.689941,5270680000
2010-01-08,1140.520020,1145.390015,1136.219971,1144.979980,1144.979980,4389590000


In [13]:
print('Total dataset has {} samples, and {} features.'.format(df.shape[0], \
                                                              df.shape[1]))

Total dataset has 3430 samples, and 6 features.


# 2. Feature engineering

In [14]:
FE_df = df

In [15]:
len(FE_df)

3430

# 3. Splits test 

In [16]:
win_size = 60
num_epoch = 50
num_cell = 50
batch_size = 64
dropout = 0.5

## 3.1. Splits from begining 

In [35]:
sp1_performance_list = []

In [36]:
#Splitting to Training set and Test set
nb_splits = 10
timesplit= TimeSeriesSplit(n_splits=nb_splits)

In [37]:
data_training = []
data_testing = []
# Parcourez les splits générés par TimeSeriesSplit
for train_index, test_index in timesplit.split(FE_df):
    data_training.append(FE_df.iloc[train_index])
    data_testing.append(FE_df.iloc[test_index])

In [38]:
for i in range(nb_splits):
    train_shape = data_training[i].shape
    test_shape = data_testing[i].shape
    print(f"Split {i+1}: Training shape - {train_shape}, Testing shape - {test_shape}")

Split 1: Training shape - (320, 6), Testing shape - (311, 6)
Split 2: Training shape - (631, 6), Testing shape - (311, 6)
Split 3: Training shape - (942, 6), Testing shape - (311, 6)
Split 4: Training shape - (1253, 6), Testing shape - (311, 6)
Split 5: Training shape - (1564, 6), Testing shape - (311, 6)
Split 6: Training shape - (1875, 6), Testing shape - (311, 6)
Split 7: Training shape - (2186, 6), Testing shape - (311, 6)
Split 8: Training shape - (2497, 6), Testing shape - (311, 6)
Split 9: Training shape - (2808, 6), Testing shape - (311, 6)
Split 10: Training shape - (3119, 6), Testing shape - (311, 6)


In [39]:
data_testing[4].head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2016-03-22,2048.639893,2056.600098,2040.569946,2049.800049,2049.800049,3418460000
2016-03-23,2048.550049,2048.550049,2034.859985,2036.709961,2036.709961,3639510000
2016-03-24,2032.479980,2036.040039,2022.489990,2035.939941,2035.939941,3407720000
2016-03-28,2037.890015,2042.670044,2031.959961,2037.050049,2037.050049,2809090000
2016-03-29,2035.750000,2055.909912,2028.310059,2055.010010,2055.010010,3822330000


In [40]:
for i in range(nb_splits):
    resultat = hyper_GRU_M0(data_training[i], data_testing[i], win_size, num_epoch, num_cell, batch_size, dropout)
    sp1_performance_list.append(resultat)

Epoch 1/50
5/5 [==============================] - 2s 110ms/step - loss: 0.3634 - val_loss: 0.1919
Epoch 2/50
5/5 [==============================] - 0s 30ms/step - loss: 0.1218 - val_loss: 0.0220
Epoch 3/50
5/5 [==============================] - 0s 30ms/step - loss: 0.0691 - val_loss: 0.0197
Epoch 4/50
5/5 [==============================] - 0s 31ms/step - loss: 0.0969 - val_loss: 0.0333
Epoch 5/50
5/5 [==============================] - 0s 26ms/step - loss: 0.0854 - val_loss: 0.0236
Epoch 6/50
5/5 [==============================] - 0s 30ms/step - loss: 0.0717 - val_loss: 0.0087
Epoch 7/50
5/5 [==============================] - 0s 24ms/step - loss: 0.0535 - val_loss: 0.0067
Epoch 8/50
5/5 [==============================] - 0s 24ms/step - loss: 0.0586 - val_loss: 0.0076
Epoch 9/50
5/5 [==============================] - 0s 28ms/step - loss: 0.0448 - val_loss: 0.0123
Epoch 10/50
5/5 [==============================] - 0s 27ms/step - loss: 0.0409 - val_loss: 0.0132
Epoch 11/50
5/5 [===========

Epoch 35/50
9/9 [==============================] - 0s 21ms/step - loss: 0.0096 - val_loss: 0.0011
Epoch 36/50
9/9 [==============================] - 0s 22ms/step - loss: 0.0106 - val_loss: 0.0027
Epoch 37/50
9/9 [==============================] - 0s 24ms/step - loss: 0.0089 - val_loss: 0.0011
Epoch 38/50
9/9 [==============================] - 0s 23ms/step - loss: 0.0082 - val_loss: 0.0032
Epoch 39/50
9/9 [==============================] - 0s 21ms/step - loss: 0.0100 - val_loss: 0.0010
Epoch 40/50
9/9 [==============================] - 0s 24ms/step - loss: 0.0099 - val_loss: 0.0022
Epoch 41/50
9/9 [==============================] - 0s 23ms/step - loss: 0.0086 - val_loss: 0.0015
Epoch 42/50
9/9 [==============================] - 0s 24ms/step - loss: 0.0078 - val_loss: 0.0014
Epoch 43/50
9/9 [==============================] - 0s 23ms/step - loss: 0.0104 - val_loss: 0.0015
Epoch 44/50
9/9 [==============================] - 0s 25ms/step - loss: 0.0096 - val_loss: 0.0013
Epoch 45/50
9/9 [===

19/19 [==============================] - 0s 21ms/step - loss: 0.0050 - val_loss: 0.0065
Epoch 18/50
19/19 [==============================] - 0s 21ms/step - loss: 0.0050 - val_loss: 0.0070
Epoch 19/50
19/19 [==============================] - 0s 20ms/step - loss: 0.0048 - val_loss: 0.0094
Epoch 20/50
19/19 [==============================] - 0s 20ms/step - loss: 0.0050 - val_loss: 0.0061
Epoch 21/50
19/19 [==============================] - 0s 20ms/step - loss: 0.0046 - val_loss: 0.0062
Epoch 22/50
19/19 [==============================] - 0s 21ms/step - loss: 0.0045 - val_loss: 0.0062
Epoch 23/50
19/19 [==============================] - 0s 21ms/step - loss: 0.0043 - val_loss: 0.0061
Epoch 24/50
19/19 [==============================] - 0s 21ms/step - loss: 0.0046 - val_loss: 0.0060
Epoch 25/50
19/19 [==============================] - 0s 21ms/step - loss: 0.0050 - val_loss: 0.0060
Epoch 26/50
19/19 [==============================] - 0s 20ms/step - loss: 0.0039 - val_loss: 0.0064
Epoch 27/50


24/24 [==============================] - 0s 19ms/step - loss: 0.0048 - val_loss: 7.5177e-04
Epoch 49/50
24/24 [==============================] - 0s 19ms/step - loss: 0.0043 - val_loss: 8.2141e-04
Epoch 50/50
24/24 [==============================] - 0s 19ms/step - loss: 0.0043 - val_loss: 5.1878e-04
Epoch 1/50
29/29 [==============================] - 3s 31ms/step - loss: 0.0245 - val_loss: 0.0125
Epoch 2/50
29/29 [==============================] - 1s 19ms/step - loss: 0.0150 - val_loss: 0.0029
Epoch 3/50
29/29 [==============================] - 1s 33ms/step - loss: 0.0110 - val_loss: 0.0059
Epoch 4/50
29/29 [==============================] - 1s 24ms/step - loss: 0.0092 - val_loss: 0.0025
Epoch 5/50
29/29 [==============================] - 1s 22ms/step - loss: 0.0084 - val_loss: 0.0022
Epoch 6/50
29/29 [==============================] - 1s 21ms/step - loss: 0.0065 - val_loss: 0.0017
Epoch 7/50
29/29 [==============================] - 1s 20ms/step - loss: 0.0072 - val_loss: 0.0022
Epoch 8

34/34 [==============================] - 1s 21ms/step - loss: 0.0036 - val_loss: 0.0028
Epoch 31/50
34/34 [==============================] - 1s 20ms/step - loss: 0.0033 - val_loss: 0.0016
Epoch 32/50
34/34 [==============================] - 1s 21ms/step - loss: 0.0031 - val_loss: 0.0015
Epoch 33/50
34/34 [==============================] - 1s 20ms/step - loss: 0.0027 - val_loss: 0.0020
Epoch 34/50
34/34 [==============================] - 1s 20ms/step - loss: 0.0030 - val_loss: 0.0016
Epoch 35/50
34/34 [==============================] - 1s 26ms/step - loss: 0.0025 - val_loss: 0.0016
Epoch 36/50
34/34 [==============================] - 1s 22ms/step - loss: 0.0028 - val_loss: 0.0016
Epoch 37/50
34/34 [==============================] - 1s 22ms/step - loss: 0.0031 - val_loss: 0.0037
Epoch 38/50
34/34 [==============================] - 1s 21ms/step - loss: 0.0029 - val_loss: 0.0019
Epoch 39/50
34/34 [==============================] - 1s 22ms/step - loss: 0.0029 - val_loss: 0.0040
Epoch 40/50


43/43 [==============================] - 1s 19ms/step - loss: 0.0035 - val_loss: 0.0023
Epoch 13/50
43/43 [==============================] - 1s 19ms/step - loss: 0.0031 - val_loss: 0.0023
Epoch 14/50
43/43 [==============================] - 1s 19ms/step - loss: 0.0035 - val_loss: 0.0032
Epoch 15/50
43/43 [==============================] - 1s 19ms/step - loss: 0.0033 - val_loss: 0.0021
Epoch 16/50
43/43 [==============================] - 1s 19ms/step - loss: 0.0032 - val_loss: 0.0024
Epoch 17/50
43/43 [==============================] - 1s 19ms/step - loss: 0.0028 - val_loss: 0.0022
Epoch 18/50
43/43 [==============================] - 1s 19ms/step - loss: 0.0030 - val_loss: 0.0021
Epoch 19/50
43/43 [==============================] - 1s 19ms/step - loss: 0.0026 - val_loss: 0.0021
Epoch 20/50
43/43 [==============================] - 1s 19ms/step - loss: 0.0025 - val_loss: 0.0024
Epoch 21/50
43/43 [==============================] - 1s 19ms/step - loss: 0.0026 - val_loss: 0.0023
Epoch 22/50


48/48 [==============================] - 1s 17ms/step - loss: 0.0017 - val_loss: 0.0064
Epoch 45/50
48/48 [==============================] - 1s 17ms/step - loss: 0.0020 - val_loss: 0.0033
Epoch 46/50
48/48 [==============================] - 1s 18ms/step - loss: 0.0019 - val_loss: 0.0038
Epoch 47/50
48/48 [==============================] - 1s 19ms/step - loss: 0.0016 - val_loss: 0.0032
Epoch 48/50
48/48 [==============================] - 1s 19ms/step - loss: 0.0018 - val_loss: 0.0041
Epoch 49/50
48/48 [==============================] - 1s 19ms/step - loss: 0.0018 - val_loss: 0.0042
Epoch 50/50
48/48 [==============================] - 1s 19ms/step - loss: 0.0021 - val_loss: 0.0038


In [41]:
sp1_performance = pd.DataFrame(sp1_performance_list, columns=['Modele', 'win_size', \
                'num_cell', 'num_epoch', 'batch_size', 'dropout', 'rsme', 'duration(s)'])
sp1_performance.to_csv('sp1_performance.csv')

## 3.2 Split windows with constant training

In [42]:
sp2_performance_list = []

In [49]:
#Splitting to Training set and Test set
nb_splits = 30
timesplit= TimeSeriesSplit(n_splits=nb_splits, max_train_size=300 ,test_size=100)

In [50]:
data_training = []
data_testing = []
# Parcourez les splits générés par TimeSeriesSplit
for train_index, test_index in timesplit.split(FE_df):
    data_training.append(FE_df.iloc[train_index])
    data_testing.append(FE_df.iloc[test_index])

In [51]:
for i in range(nb_splits):
    train_shape = data_training[i].shape
    test_shape = data_testing[i].shape
    print(f"Split {i+1}: Training shape - {train_shape}, Testing shape - {test_shape}")

Split 1: Training shape - (300, 6), Testing shape - (100, 6)
Split 2: Training shape - (300, 6), Testing shape - (100, 6)
Split 3: Training shape - (300, 6), Testing shape - (100, 6)
Split 4: Training shape - (300, 6), Testing shape - (100, 6)
Split 5: Training shape - (300, 6), Testing shape - (100, 6)
Split 6: Training shape - (300, 6), Testing shape - (100, 6)
Split 7: Training shape - (300, 6), Testing shape - (100, 6)
Split 8: Training shape - (300, 6), Testing shape - (100, 6)
Split 9: Training shape - (300, 6), Testing shape - (100, 6)
Split 10: Training shape - (300, 6), Testing shape - (100, 6)
Split 11: Training shape - (300, 6), Testing shape - (100, 6)
Split 12: Training shape - (300, 6), Testing shape - (100, 6)
Split 13: Training shape - (300, 6), Testing shape - (100, 6)
Split 14: Training shape - (300, 6), Testing shape - (100, 6)
Split 15: Training shape - (300, 6), Testing shape - (100, 6)
Split 16: Training shape - (300, 6), Testing shape - (100, 6)
Split 17: Trainin

In [52]:
for i in range(nb_splits):
    resultat = hyper_GRU_M0(data_training[i], data_testing[i], win_size, num_epoch, num_cell, batch_size, dropout)
    sp2_performance_list.append(resultat)

Epoch 1/50
4/4 [==============================] - 2s 133ms/step - loss: 1.0149 - val_loss: 0.3907
Epoch 2/50
4/4 [==============================] - 0s 32ms/step - loss: 0.4751 - val_loss: 0.1281
Epoch 3/50
4/4 [==============================] - 0s 32ms/step - loss: 0.1703 - val_loss: 0.0226
Epoch 4/50
4/4 [==============================] - 0s 32ms/step - loss: 0.0787 - val_loss: 0.0216
Epoch 5/50
4/4 [==============================] - 0s 31ms/step - loss: 0.1021 - val_loss: 0.0446
Epoch 6/50
4/4 [==============================] - 0s 29ms/step - loss: 0.1116 - val_loss: 0.0400
Epoch 7/50
4/4 [==============================] - 0s 31ms/step - loss: 0.0963 - val_loss: 0.0245
Epoch 8/50
4/4 [==============================] - 0s 30ms/step - loss: 0.0771 - val_loss: 0.0145
Epoch 9/50
4/4 [==============================] - 0s 31ms/step - loss: 0.0683 - val_loss: 0.0124
Epoch 10/50
4/4 [==============================] - 0s 28ms/step - loss: 0.0632 - val_loss: 0.0138
Epoch 11/50
4/4 [===========

Epoch 35/50
4/4 [==============================] - 0s 24ms/step - loss: 0.0246 - val_loss: 0.0041
Epoch 36/50
4/4 [==============================] - 0s 30ms/step - loss: 0.0246 - val_loss: 0.0038
Epoch 37/50
4/4 [==============================] - 0s 34ms/step - loss: 0.0243 - val_loss: 0.0038
Epoch 38/50
4/4 [==============================] - 0s 29ms/step - loss: 0.0225 - val_loss: 0.0037
Epoch 39/50
4/4 [==============================] - 0s 29ms/step - loss: 0.0229 - val_loss: 0.0038
Epoch 40/50
4/4 [==============================] - 0s 31ms/step - loss: 0.0189 - val_loss: 0.0045
Epoch 41/50
4/4 [==============================] - 0s 32ms/step - loss: 0.0226 - val_loss: 0.0039
Epoch 42/50
4/4 [==============================] - 0s 33ms/step - loss: 0.0240 - val_loss: 0.0036
Epoch 43/50
4/4 [==============================] - 0s 31ms/step - loss: 0.0215 - val_loss: 0.0037
Epoch 44/50
4/4 [==============================] - 0s 34ms/step - loss: 0.0237 - val_loss: 0.0037
Epoch 45/50
4/4 [===

Epoch 19/50
4/4 [==============================] - 0s 23ms/step - loss: 0.0344 - val_loss: 0.0049
Epoch 20/50
4/4 [==============================] - 0s 24ms/step - loss: 0.0329 - val_loss: 0.0047
Epoch 21/50
4/4 [==============================] - 0s 26ms/step - loss: 0.0314 - val_loss: 0.0050
Epoch 22/50
4/4 [==============================] - 0s 24ms/step - loss: 0.0251 - val_loss: 0.0053
Epoch 23/50
4/4 [==============================] - 0s 25ms/step - loss: 0.0294 - val_loss: 0.0055
Epoch 24/50
4/4 [==============================] - 0s 26ms/step - loss: 0.0285 - val_loss: 0.0051
Epoch 25/50
4/4 [==============================] - 0s 26ms/step - loss: 0.0240 - val_loss: 0.0045
Epoch 26/50
4/4 [==============================] - 0s 29ms/step - loss: 0.0253 - val_loss: 0.0045
Epoch 27/50
4/4 [==============================] - 0s 24ms/step - loss: 0.0255 - val_loss: 0.0046
Epoch 28/50
4/4 [==============================] - 0s 25ms/step - loss: 0.0271 - val_loss: 0.0042
Epoch 29/50
4/4 [===

4/4 [==============================] - 0s 25ms/step - loss: 0.1362 - val_loss: 0.0404
Epoch 3/50
4/4 [==============================] - 0s 30ms/step - loss: 0.0814 - val_loss: 0.0091
Epoch 4/50
4/4 [==============================] - 0s 26ms/step - loss: 0.1000 - val_loss: 0.0251
Epoch 5/50
4/4 [==============================] - 0s 22ms/step - loss: 0.1035 - val_loss: 0.0149
Epoch 6/50
4/4 [==============================] - 0s 24ms/step - loss: 0.0761 - val_loss: 0.0054
Epoch 7/50
4/4 [==============================] - 0s 29ms/step - loss: 0.0566 - val_loss: 0.0134
Epoch 8/50
4/4 [==============================] - 0s 36ms/step - loss: 0.0645 - val_loss: 0.0203
Epoch 9/50
4/4 [==============================] - 0s 24ms/step - loss: 0.0543 - val_loss: 0.0162
Epoch 10/50
4/4 [==============================] - 0s 31ms/step - loss: 0.0639 - val_loss: 0.0084
Epoch 11/50
4/4 [==============================] - 0s 26ms/step - loss: 0.0497 - val_loss: 0.0047
Epoch 12/50
4/4 [======================

Epoch 36/50
4/4 [==============================] - 0s 27ms/step - loss: 0.0136 - val_loss: 0.0070
Epoch 37/50
4/4 [==============================] - 0s 24ms/step - loss: 0.0142 - val_loss: 0.0091
Epoch 38/50
4/4 [==============================] - 0s 28ms/step - loss: 0.0171 - val_loss: 0.0081
Epoch 39/50
4/4 [==============================] - 0s 26ms/step - loss: 0.0123 - val_loss: 0.0058
Epoch 40/50
4/4 [==============================] - 0s 24ms/step - loss: 0.0161 - val_loss: 0.0053
Epoch 41/50
4/4 [==============================] - 0s 26ms/step - loss: 0.0115 - val_loss: 0.0060
Epoch 42/50
4/4 [==============================] - 0s 26ms/step - loss: 0.0123 - val_loss: 0.0071
Epoch 43/50
4/4 [==============================] - 0s 26ms/step - loss: 0.0145 - val_loss: 0.0068
Epoch 44/50
4/4 [==============================] - 0s 26ms/step - loss: 0.0140 - val_loss: 0.0057
Epoch 45/50
4/4 [==============================] - 0s 27ms/step - loss: 0.0154 - val_loss: 0.0052
Epoch 46/50
4/4 [===

...

Epoch 39/50
4/4 [==============================] - 0s 26ms/step - loss: 0.0122 - val_loss: 0.0107
Epoch 40/50
4/4 [==============================] - 0s 27ms/step - loss: 0.0136 - val_loss: 0.0097
Epoch 41/50
4/4 [==============================] - 0s 24ms/step - loss: 0.0118 - val_loss: 0.0095
Epoch 42/50
4/4 [==============================] - 0s 26ms/step - loss: 0.0114 - val_loss: 0.0097
Epoch 43/50
4/4 [==============================] - 0s 24ms/step - loss: 0.0119 - val_loss: 0.0108
Epoch 44/50
4/4 [==============================] - 0s 25ms/step - loss: 0.0119 - val_loss: 0.0103
Epoch 45/50
4/4 [==============================] - 0s 24ms/step - loss: 0.0110 - val_loss: 0.0089
Epoch 46/50
4/4 [==============================] - 0s 25ms/step - loss: 0.0127 - val_loss: 0.0085
Epoch 47/50
4/4 [==============================] - 0s 21ms/step - loss: 0.0093 - val_loss: 0.0086
Epoch 48/50
4/4 [==============================] - 0s 26ms/step - loss: 0.0109 - val_loss: 0.0094
Epoch 49/50
4/4 [===

Epoch 23/50
4/4 [==============================] - 0s 30ms/step - loss: 0.0219 - val_loss: 0.0053
Epoch 24/50
4/4 [==============================] - 0s 25ms/step - loss: 0.0214 - val_loss: 0.0044
Epoch 25/50
4/4 [==============================] - 0s 30ms/step - loss: 0.0210 - val_loss: 0.0040
Epoch 26/50
4/4 [==============================] - 0s 31ms/step - loss: 0.0192 - val_loss: 0.0038
Epoch 27/50
4/4 [==============================] - 0s 26ms/step - loss: 0.0213 - val_loss: 0.0043
Epoch 28/50
4/4 [==============================] - 0s 24ms/step - loss: 0.0208 - val_loss: 0.0047
Epoch 29/50
4/4 [==============================] - 0s 27ms/step - loss: 0.0166 - val_loss: 0.0045
Epoch 30/50
4/4 [==============================] - 0s 26ms/step - loss: 0.0194 - val_loss: 0.0040
Epoch 31/50
4/4 [==============================] - 0s 29ms/step - loss: 0.0144 - val_loss: 0.0037
Epoch 32/50
4/4 [==============================] - 0s 24ms/step - loss: 0.0175 - val_loss: 0.0034
Epoch 33/50
4/4 [===

4/4 [==============================] - 0s 29ms/step - loss: 0.0515 - val_loss: 0.0170
Epoch 7/50
4/4 [==============================] - 0s 24ms/step - loss: 0.0553 - val_loss: 0.0174
Epoch 8/50
4/4 [==============================] - 0s 30ms/step - loss: 0.0502 - val_loss: 0.0105
Epoch 9/50
4/4 [==============================] - 0s 23ms/step - loss: 0.0343 - val_loss: 0.0067
Epoch 10/50
4/4 [==============================] - 0s 30ms/step - loss: 0.0397 - val_loss: 0.0065
Epoch 11/50
4/4 [==============================] - 0s 24ms/step - loss: 0.0308 - val_loss: 0.0066
Epoch 12/50
4/4 [==============================] - 0s 29ms/step - loss: 0.0382 - val_loss: 0.0061
Epoch 13/50
4/4 [==============================] - 0s 28ms/step - loss: 0.0292 - val_loss: 0.0074
Epoch 14/50
4/4 [==============================] - 0s 26ms/step - loss: 0.0288 - val_loss: 0.0086
Epoch 15/50
4/4 [==============================] - 0s 24ms/step - loss: 0.0255 - val_loss: 0.0079
Epoch 16/50
4/4 [==================

Epoch 40/50
4/4 [==============================] - 0s 25ms/step - loss: 0.0113 - val_loss: 0.0094
Epoch 41/50
4/4 [==============================] - 0s 30ms/step - loss: 0.0126 - val_loss: 0.0088
Epoch 42/50
4/4 [==============================] - 0s 33ms/step - loss: 0.0100 - val_loss: 0.0085
Epoch 43/50
4/4 [==============================] - 0s 27ms/step - loss: 0.0116 - val_loss: 0.0085
Epoch 44/50
4/4 [==============================] - 0s 25ms/step - loss: 0.0108 - val_loss: 0.0089
Epoch 45/50
4/4 [==============================] - 0s 23ms/step - loss: 0.0086 - val_loss: 0.0087
Epoch 46/50
4/4 [==============================] - 0s 25ms/step - loss: 0.0109 - val_loss: 0.0084
Epoch 47/50
4/4 [==============================] - 0s 27ms/step - loss: 0.0091 - val_loss: 0.0082
Epoch 48/50
4/4 [==============================] - 0s 30ms/step - loss: 0.0103 - val_loss: 0.0081
Epoch 49/50
4/4 [==============================] - 0s 30ms/step - loss: 0.0105 - val_loss: 0.0083
Epoch 50/50
4/4 [===

Epoch 24/50
4/4 [==============================] - 0s 29ms/step - loss: 0.0246 - val_loss: 0.0063
Epoch 25/50
4/4 [==============================] - 0s 26ms/step - loss: 0.0225 - val_loss: 0.0063
Epoch 26/50
4/4 [==============================] - 0s 27ms/step - loss: 0.0248 - val_loss: 0.0063
Epoch 27/50
4/4 [==============================] - 0s 29ms/step - loss: 0.0200 - val_loss: 0.0064
Epoch 28/50
4/4 [==============================] - 0s 23ms/step - loss: 0.0220 - val_loss: 0.0066
Epoch 29/50
4/4 [==============================] - 0s 29ms/step - loss: 0.0238 - val_loss: 0.0064
Epoch 30/50
4/4 [==============================] - 0s 25ms/step - loss: 0.0173 - val_loss: 0.0063
Epoch 31/50
4/4 [==============================] - 0s 29ms/step - loss: 0.0176 - val_loss: 0.0064
Epoch 32/50
4/4 [==============================] - 0s 25ms/step - loss: 0.0216 - val_loss: 0.0065
Epoch 33/50
4/4 [==============================] - 0s 26ms/step - loss: 0.0228 - val_loss: 0.0068
Epoch 34/50
4/4 [===

In [53]:
sp2_performance = pd.DataFrame(sp2_performance_list, columns=['Modele', 'win_size', \
                'num_cell', 'num_epoch', 'batch_size', 'dropout', 'rsme', 'duration(s)'])
sp2_performance.to_csv('sp2_performance.csv')